# Guide: Tracking QUBO experiments

In this guide, we will show you how to track QUBO experiments solved by QAOA and VQE.

In [1]:
from qiskit.algorithms.minimum_eigensolvers import QAOA, SamplingVQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.circuit.library import TwoLocal
from qiskit_optimization import QuadraticProgram

from purplecaffeine import Trial, LocalStorage

First let's create a simple QUBO problem.

In [2]:
qubo = QuadraticProgram(name="qubo_trial")
qubo.binary_var("x")
qubo.binary_var("y")
qubo.binary_var("z")
qubo.minimize(linear=[1, -2, 3], quadratic={("x", "y"): 1, ("x", "z"): -1, ("y", "z"): 2})
print(qubo.prettyprint())

Problem name: qubo_trial

Minimize
  x*y - x*z + 2*y*z + x - 2*y + 3*z

Subject to
  No constraints

  Binary variables (3)
    x y z



## QAOA

We setup our experiment using QAOA.

Now let's register this experiment in `LocalStorage`

In [3]:
local_storage = LocalStorage("./trials")

In [4]:
with Trial("QAOA trial", storage=local_storage) as trial:
    # track some parameters
    trial.add_parameter("algo", "qaoa")
    trial.add_parameter("sampler", "qiskit.primitives.Sampler")
    trial.add_parameter("optimizer", "qiskit.algorithms.optimizers.COBYLA")
    
    # track usefull data
    trial.add_text("qubo", qubo.export_as_lp_string())
    qaoa_mes = QAOA(
        sampler=Sampler(),
        optimizer=COBYLA(),
        callback=lambda idx, params, mean, std: trial.add_metric("qaoa_history", mean)
    )
    
    # run
    qaoa = MinimumEigenOptimizer(qaoa_mes)
    qaoa_result = qaoa.solve(qubo)
    
    # track results of run
    trial.add_text("qaoa_result", qaoa_result.prettyprint())

## VQE

We setup the ansatz circuit experiment for the VQE.

In [5]:
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")

Now let's track it too and save it in our `LocalStorage` as well.

In [6]:
with Trial("VQE trial", storage=local_storage) as trial:
    # track some parameters
    trial.add_parameter("algo", "vqe")
    trial.add_parameter("sampler", "qiskit.primitives.Sampler")
    trial.add_parameter("optimizer", "qiskit.algorithms.optimizers.COBYLA")
    
    # track usefull data
    trial.add_text("qubo", qubo.export_as_lp_string())
    
    # track some objects
    trial.add_circuit("ansatz", ansatz)
    
    vqe_mes = SamplingVQE(
        sampler=Sampler(),
        ansatz=ansatz,
        optimizer=COBYLA(),
        callback=lambda idx, params, mean, std: trial.add_metric("vqe_history", mean)
    )
    
    # run
    vqe = MinimumEigenOptimizer(vqe_mes)
    vqe_result = vqe.solve(qubo)
    
    # track results of run
    trial.add_text("vqe_result", vqe_result.prettyprint())

And now we can see both our experiments in our `Storage`:

In [7]:
local_storage.list()

[<Trial [QAOA trial] 671b7bc5-58c2-4162-8756-bddca2a71548>,
 <Trial [VQE trial] e37d36e9-f9b0-4721-8b85-ad8b1784e9ed>]